- w is the phenotype distribution on plate $x(T^\tau)$
- v is the phenotype distribution on plate $x(\hat{T}-1)$
- $\phi$ skews the infection decisions for each plate.
- $\phi_i w_i = v_i$
- The aim of this script is to find out all $\phi$ for all plates
- each well phenotype i has a propability $\alpha_i$ to infect

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from evaluation_scripts.base import get_pathes, dict_to_json, load_toml
from evaluation_scripts.well_ident import build_dictionaries
from itertools import product as iterprod
import json
from model_scripts.decisions_04 import get_x_tau, get_number_of_received_infections, get_phi, get_alpha

experiments = ["20220412", "20220127", "20210417"]
regular_phenos = ["UI", "S", "A_r", "B_r", "A&B", "AB_r"] 


In [ ]:
for exp in experiments:
    print(exp)
    ## LOAD
    jupyter_path, base_path, path, analysis_path, os = get_pathes(exp + "_output")
    hue_infos, strategies, plate_rows, agar_strain_dict = build_dictionaries()
    Data = pd.read_pickle(analysis_path+os.sep+"obj"+os.sep+"01b_Data.pkl")
    with open(os.path.join(path, "exp_pars.json")) as json_data:
            exp_pars = json.load(json_data)
            
    ## DO
    # Calulate Phi
    Data["x_tau"] = Data.apply(lambda x: get_x_tau(x), axis = 1)
    Data["infected_by_n"] = Data.apply(lambda x: get_number_of_received_infections(x), axis = 1)
    cols = ["transfer_n", "strategy", "x_tau", "n"]
    Data["n"] = True
    summary = Data[Data.x_tau.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
    cols = ["transfer_n", "strategy", "x_tau", "infected_by_n"]
    v = Data[Data.x_tau.isin(regular_phenos)][cols].groupby(cols[:-1]).sum().unstack(fill_value=0).stack()
    summary = pd.concat([summary, v],axis=1)
    t = Data.transfer_n.unique()
    for x in iterprod(t, strategies.values()):
        summary.loc[x, "w"] = list(summary.loc[x, "n"]/summary.loc[x,"n"].sum())
        summary.loc[x, "v"] = list(summary.loc[x, "infected_by_n"]/summary.loc[x,"infected_by_n"].sum())
    summary["phi"]  = summary.apply(lambda x: get_phi(x), axis = 1)
    
    # Calculate Alpha
    Data["infects"] = (Data.infection_to_well_id.isnull()==False)
    cols = ["transfer_n", "strategy", "phenotype", "infects"]
    summary["infects"] = Data[Data.x_tau.isin(regular_phenos)][cols].groupby(cols[:-1]).sum().unstack(fill_value=0).stack()
    Data["m"] = True
    cols = ["transfer_n", "strategy", "phenotype", "m"]
    summary["m"] = Data[Data.x_tau.isin(regular_phenos)][cols].groupby(cols[:-1]).sum().unstack(fill_value=0).stack()
    summary["alpha"] = summary.apply(lambda x: get_alpha(x, exp_pars["rates"][0]["infection"]), axis = 1)
    
    
    ## SAVE
    summary.to_pickle(os.path.join(analysis_path, "obj", "04a_decisions.pkl"))

20220412


In [ ]:
summary